In [70]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sb

In [71]:
df = pd.read_csv("Filtered KNBS_data.csv")

/tmp/ipykernel_5830/1630855121.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Filtered KNBS_data.csv")


In [72]:
df.head(10)

,b04,b05_mm,b05_yy,b06_yy,b14,c10_l,c10_g,c11,d02_1,d02_2,...,e22_2,e22_3,e23_1,e23_2,e23_3,g06,g07,g08,g09,g10
0,Female,NaN,31,1984,Catholic,NaN,NaN,NaN,No,Yes,...,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No
1,Male,NaN,9,2006,Catholic,Primary,2.0,None,No,No,...,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No
2,Male,NaN,6,2009,Catholic,Pre-primary,2.0,None,No,No,...,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No
3,Male,0.0,3,2012,Catholic,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No
4,Male,6.0,0,2015,Catholic,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Female,NaN,38,1977,Traditionalist,NaN,NaN,NaN,No,No,...,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No
6,Female,NaN,17,1998,Catholic,Primary,7.0,Cpe/kcpe,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,No,No
7,Female,NaN,16,1999,Catholic,Primary,2.0,NaN,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,No,No
8,Female,NaN,13,2002,Catholic,NaN,NaN,NaN,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,No,No
9,Female,NaN,10,2005,Catholic,NaN,NaN,NaN,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,No,No


In [73]:
df.shape

(92846, 34)

In [74]:
df.isnull().sum()

b04           0
b05_mm    80156
b05_yy        0
b06_yy        2
b14           0
c10_l     23291
c10_g     23493
c11       23297
d02_1     12807
d02_2     12807
d02_3     12807
d02_4     12807
d02_5     12807
d02_6     12807
d04_1     51186
d04_2     92789
d04_3     92839
d11_1     52395
d11_2     92568
d11_3     92768
d12a      54047
d12b      56286
d31       53609
e22_1       175
e22_2     92519
e22_3     92793
e23_1     90386
e23_2     92671
e23_3     92809
g06        7788
g07        7723
g08        7723
g09        7713
g10        7733
dtype: int64

Dropping rows with more than **80%** missing values

In [75]:
# Calculate the percentage of missing values in each row
row_missing = df.isnull().mean(axis=1)

# Drop rows that have 80% or more missing values
df = df[row_missing < 0.8]



Drop coluns with more than 80% missing values

In [76]:
# Calculate the percentage of missing values in each column
col_missing = df.isnull().mean()

# Drop columns that have 80% or more missing values
df = df.drop(col_missing[col_missing >= 0.8].index, axis=1)


In [77]:
df.isnull().sum()

b04           0
b05_yy        0
b06_yy        2
b14           0
c10_l     15783
c10_g     15985
c11       15789
d02_1      5299
d02_2      5299
d02_3      5299
d02_4      5299
d02_5      5299
d02_6      5299
d04_1     43678
d11_1     44887
d12a      46539
d12b      48778
d31       46101
e22_1       138
g06         280
g07         215
g08         215
g09         205
g10         225
dtype: int64

Creating a function to fill missing values propotionally

In [78]:
# function to fill missing values propotionally

def fill_missing_values_proportionally(df, column):
    value_counts = df[column].value_counts(normalize=True)
    def fill_missing(value):
        if pd.isnull(value):
            return np.random.choice(value_counts.index, p=value_counts.values)
        else:
            return value
    df[column] = df[column].apply(fill_missing)
    return df

In [79]:
#apply function to all columns 
missing_columns = df.columns[df.isnull().any()].tolist()

for column in missing_columns:
    df = fill_missing_values_proportionally(df, column)

In [80]:
df.isnull().sum()

b04       0
b05_yy    0
b06_yy    0
b14       0
c10_l     0
c10_g     0
c11       0
d02_1     0
d02_2     0
d02_3     0
d02_4     0
d02_5     0
d02_6     0
d04_1     0
d11_1     0
d12a      0
d12b      0
d31       0
e22_1     0
g06       0
g07       0
g08       0
g09       0
g10       0
dtype: int64

### Feature Engineering

In [81]:
#selecting dependet and independet variables

dep_variable = df["d11_1"]
ind_variable  = df.drop(columns=["d11_1"])

##### Converting to Continous variables

In [82]:
#selecting object types
non_numeric_columns = ind_variable.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
numeric_columns = ind_variable.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [85]:
#converting to continous
ind_variables = pd.get_dummies(ind_variable, columns=non_numeric_columns, drop_first=True)

#### Data Imbalance

In [84]:
#import modules
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [89]:
# Dependet and independet variable
y = dep_variable.copy()
X = ind_variables.copy()


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# applying smote to the training data
smote = SMOTE(sampling_strategy="auto", random_state=0)
